# Fase 2: Clasificación de la ejecución del movimiento
**Descripción**\
En esta segunda fase del proyecto, el objetivo es evaluar la correcta ejecución de los gestos identificados en la Fase 1. Después de haber determinado el mejor modelo para reconocer el tipo de gesto realizado por el paciente, ahora buscamos identificar si cada gesto se realiza correctamente o no.\
Este notebook aborda los siguientes aspectos:
* Creación de modelos: para cada tipo de gesto identificado en la Fase 1, se desarrollan modelos de clasificación separados, para determinar si un gesto es ejecutado de manera correcta o incorrecta
* Evaluación de la ejecución.

**Entrada**
* Datos de gestos procesados en el archivo ``medidasPerRepetition.csv``.

**Salida**
* Los resultados detallados del ajuste de modelos tras aplicar varias técnica de balanceo de datos se almacenan en el archivo ``Results_imblearn.txt``. 

**Índice**
1. [Preprocesado](#1-preprocesado)
2. [Funciones adicionales](#2-funciones-adicionales)
3. [Balanceo de datos y modelos de clasificación](#3-balanceo-de-datos-y-modelos-de-clasificación)
4. [Pipeline](#4-pipeline)
****

In [1]:
# ---------- importar librerías---------
# Manipular los datos
import pandas as pd
import numpy as np

# Gráficas
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
sns.set_theme(style="darkgrid")

# scikit-learn (ML en python)
## Procesar el dataset
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import LeaveOneGroupOut # LeavePGroupsOut
## Modelos ML
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
## Evaluación de los modelos
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
## Hiperparametrizacion
from sklearn.model_selection import RandomizedSearchCV

# Class imbalance
from imblearn.under_sampling import NearMiss, EditedNearestNeighbours
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek, SMOTEENN
from collections import Counter

## Seleccion de variables
from sklearn.decomposition import PCA

# Pipeline
from imblearn.pipeline import Pipeline


# Para ignorar los Warnings
import warnings
from sklearn.exceptions import FitFailedWarning 

warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = UserWarning)

In [2]:
#---------Importar los datos-----------
# Dataframe medidas calculadas por repetición
df = pd.read_csv('../Resultados/medidasPerRepetition.csv', dtype=object)

*****
## 1. Preprocesado

In [3]:
# ------Preparar el dataset-------
encoder = OrdinalEncoder(categories=[list(set(df["Position"].values))])
encoder.fit(df[["Position"]])
df["Position"] = encoder.transform(df[["Position"]])

# pasar variable obj to numeric
df = df.apply(pd.to_numeric, errors='ignore')

# cmabiar las poorly executed to incorrectly executed
df[df['CorrectLabel']==3] = 2

****
## 2. Funciones adicionales

In [4]:
#-------Funciones--------

# dividir en test y train df_gestures
def split_df_gestures(df:pd.DataFrame, target: str):
  """
  Divide un DataFrame en conjuntos de entrenamiento y prueba basados en los valores de 'SubjectID'.

  Parámetros
  ----------
  df : pd.DataFrame
      DataFrame que contiene los datos completos incluyendo características y el target.
  target : str
      Nombre de la columna objetivo que contiene las etiquetas de la clase.

  Return
  -------
  X_train : pd.DataFrame
      Conjunto de datos de entrenamiento con las características (sin la columna objetivo).
  X_test : pd.DataFrame
      Conjunto de datos de prueba con las características (sin la columna objetivo).
  y_train : np.ndarray
      Array 1D con las etiquetas del conjunto de entrenamiento.
  y_test : np.ndarray
      Array 1D con las etiquetas del conjunto de prueba.
  """
  # Selecciona los datos de prueba (varios sujetos)
  test_df = df.loc[(df.SubjectID==101) | (df.SubjectID==105) | (df.SubjectID==201) | (df.SubjectID==202) | (df.SubjectID==301) | (df.SubjectID==302)]
  
  # Selecciona los datos de entrenamiento excluyendo los mismos sujetos
  train_df = df.loc[(df.SubjectID!=101) & (df.SubjectID!=105) & (df.SubjectID!=201) & (df.SubjectID!=202) & (df.SubjectID!=301) & (df.SubjectID!=302)]

  # Separa las características y las etiquetas en el conjunto de entrenamiento
  X_train = train_df.drop([target], axis=1)  
  y_train = pd.DataFrame(train_df[target])   

  # Separa las características y las etiquetas en el conjunto de prueba
  X_test = test_df.drop([target], axis=1)  
  y_test = pd.DataFrame(test_df[target])   

  # Convierte los DataFrames de etiquetas a arrays 1D
  y_train = y_train.values.ravel()
  y_test = y_test.values.ravel()

  # Devuelve los conjuntos de entrenamiento y prueba
  return X_train, X_test, y_train, y_test

# Para entrenar los modelos de ML
def aplicar_modelos(X: pd.DataFrame, y: np.ndarray, tX: pd.DataFrame, ty: np.ndarray, file):
    """
    Entrena y evalua una lista de modelos deaprendizaje automático,\\
    guardando los resultados en una archivo

    Parámetros
    ----------
    X : pd.DataFrame
        Conjunto de datos de entrenamiento con las características (variables independientes).
    y : np.ndarray
        Array con las etiquetas (valores de salida) del conjunto de entrenamiento.
    tX : pd.DataFrame
        Conjunto de datos de prueba con las características.
    ty : np.ndarray
        Array con las etiquetas verdaderas del conjunto de prueba.
    file : archivo
        Objeto de archivo abierto para escribir los resultados.
    """
    # creación de los modelos
    knn = KNeighborsClassifier()  # K-Neighbors classifier
    dtree = DecisionTreeClassifier()  # Decision tree
    nb = GaussianNB()  # Naive Bayes
    svm = SVC()  # Support Vector Machine
    rf = RandomForestClassifier()  # Random Forest
    lr = LogisticRegression()  # Logistic Regression

    models = [knn, dtree, nb, svm, rf, lr]

    # Método de validación cruzada Leave-One-Group-Out basado en SubjectID
    logo = LeaveOneGroupOut()
    grupos = X['SubjectID']

    # Iteramos sobre los modelos
    for modelo in models:
        # Evaluación de modelos mediante validación cruzada
        accuracy_scores = cross_val_score(modelo, X, y, groups=grupos, cv=logo, scoring='accuracy')
        f1_scores = cross_val_score(modelo, X, y, groups=grupos, cv=logo, scoring='f1')
        
        # Escribiendo los resultados de la validación cruzada en el archivo
        file.write(f"\nModelo: {modelo.__class__.__name__}\n")
        file.write(f"Precision media: {accuracy_scores.mean():.2f} con una desviacion estandar de {accuracy_scores.std():.2f}\n")
        file.write(f"F1-score media: {f1_scores.mean():.2f} con una desviacion estandar de {f1_scores.std():.2f}\n")

        # Evaluación final en el conjunto de prueba
        modelo.fit(X, y)  # Entrenamos el modelo con todos los datos de entrenamiento
        prediction = modelo.predict(tX)  # Predicciones en los datos de prueba
        score = f1_score(ty, prediction, average='weighted', zero_division=0) 
        file.write(f"Evaluacion en conjunto de prueba (F1-score ponderado): {score:.2f}\n")


# Aplicar imbalance learn
def aplicar_imblearn(X: pd.DataFrame, y: np.ndarray,  tX: pd.DataFrame, ty: np.ndarray, file):
    """
    Aplica una lista de técnicas de balanceo de datos para abordar el desbalance de clases.

    Parámetros
    ----------
    X : pd.DataFrame
        Conjunto de datos de entrenamiento con las características (variables independientes).
    y : np.ndarray
        Array con las etiquetas (valores de salida) del conjunto de entrenamiento.
    tX : pd.DataFrame
        Conjunto de datos de prueba con las características.
    ty : np.ndarray
        Array con las etiquetas verdaderas del conjunto de prueba.
    file : archivo
        Objeto de archivo abierto para escribir los resultados.
    """
    # Definición de las técnicas de resampling de imbalanced-learn
    nm1 = NearMiss(version=1, n_neighbors=2)  # Undersampling: NearMiss versión 1
    enn = EditedNearestNeighbours()  # Undersampling: Edited Nearest Neighbours
    adasyn = ADASYN(n_neighbors=2)  # Oversampling: Adaptive Synthetic Sampling (ADASYN)
    smote = SMOTE(k_neighbors=2)  # Oversampling: Synthetic Minority Over-sampling Technique (SMOTE)
    smotetomek = SMOTETomek(smote=smote)  # Combinación de SMOTE y Tomek links (sobremuestreo y eliminación de ruido)
    smote_enn = SMOTEENN(smote=smote)  # Combinación de SMOTE y Edited Nearest Neighbours (sobremuestreo y eliminación de ruido)

    # Lista de técnicas de resampling
    imblearns = [nm1, enn, adasyn, smote, smotetomek, smote_enn]

    # Aplicación de cada técnica de resampling
    for imblearn in imblearns:
        # Resamplear los datos
        X_im, y_im = imblearn.fit_resample(X, y)
        
        # Escribir resultados en el archivo
        file.write(f"\n\t- Tecnica de Resampling: {imblearn.__class__.__name__}\n")
        file.write(f"Distribucion de etiquetas antes del resampling: {Counter(y)}\n")
        file.write(f"Distribucion de etiquetas despues del resampling: {Counter(y_im)}\n")

        # Aplicar modelos a los datos tras resamplear
        aplicar_modelos(X_im, y_im, tX, ty, file)

****
## 3. Balanceo de datos y modelos de clasificación

In [5]:
gestures = df['GestureLabel'].unique()

# Abre el archivo 'Results_imblearn.txt' para escribir los resultados
with open('../Resultados/Results_imblearn.txt', 'w') as file:
    # Itera sobre los gestos
    for gesture in gestures:
        # Filtra el DataFrame para obtener solo las filas correspondientes al gesto
        df_gesture = df[df['GestureLabel'] == gesture]

        # Divide el DataFrame en conjuntos de entrenamiento y prueba
        X_train, X_test, y_train, y_test = split_df_gestures(df_gesture, 'CorrectLabel')  

        file.write(f'\n------------ CLASIFICACION GESTO {gesture} -----------\n')
        aplicar_imblearn(X_train, y_train, X_test, y_test, file)

Después de evaluar los resultados en el archivo de salida ``Results_imblearn.txt``, se determinó que la técnica de balanceo de datos con mejor rendimiento es **SMOTETomek**. Por lo tanto, se utilizará esta técnica para generar y presentar los nuevos datos balanceados.

In [ ]:
# -------- Aplicar SMOTETomek ---------------
smotetomek = SMOTETomek(smote=SMOTE(k_neighbors=2))

# Separar las características y las etiquetas
X = df.drop(['CorrectLabel'], axis=1) 
y = pd.DataFrame(df['CorrectLabel'])

# Resamplear los datos para balancear las clases
X_resampled, y_resampled = smotetomek.fit_resample(X, y)

balanced_data = pd.concat([X_resampled, y_resampled], axis=1)


# Calcular la cantidad de sujetos únicos por gesto y estado de ejecución
subject_count_per_gesture_correct = balanced_data.groupby(["GestureLabel", "CorrectLabel"])["SubjectID"].nunique().reset_index()

# Gráficar la distribución de sujetos después de aplicar SMOTETomek
plt.figure(figsize=(8, 5))
ax = sns.barplot(x="GestureLabel", y="SubjectID", hue="CorrectLabel",
                 data=subject_count_per_gesture_correct, palette="Paired")
# Configurar el título y las etiquetas de los ejes
ax.set_title("Distribución de los sujetos por gesto y estado de ejecución después de SMOTETomek")
ax.set_xlabel("Gesto")
ax.set_ylabel("Cantidad de sujetos")

# Configurar las etiquetas de la x-axis con los nombres de gestos 
gesture_labels = ['EFL', 'EFR', 'SFL', 'SFR', 'SAL', 'SAR', 'SFE', 'STL', 'STR']
ax.set_xticklabels(gesture_labels)

# Modificar las etiquetas de la leyenda
legend_labels = ['correctly executed', 'incorrectly executed', 'poorly executed']
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=legend_labels, title='Execution')
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

# Añadir etiquetas de conteo encima de cada barra en el gráfico
for i in ax.containers:
    ax.bar_label(i,)

plt.show()

****
## 4. Pipeline

Creacióin de pipelines que incluyen técnicas de balanceo de datos, reducción de dimensión y clasificación. Además, se emplea ``RandomizedSearchCV`` para encontrar la mejor combinación de hiperparámetros para cada pipeline.\
Los modelos optimizados se evalúan utilizando métricas de rendimiento, incluyendo la curva ROC y el área bajo la curva (AUC), para cada gesto.

In [ ]:
# Ignorar advertencias relacionadas con el ajuste del modelo
warnings.filterwarnings("ignore", category=FitFailedWarning)

# Definir la malla de parámetros
param_grid = [
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.5, 0.7, 0.8, 0.90, 1],
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': [2, 3, 5], 
        'classifier__weights': ['uniform', 'distance'], 
        'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  
    },
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.5, 0.7, 0.8, 0.90, 1],
        'classifier': [DecisionTreeClassifier()],
        'classifier__criterion': ['gini', 'entropy'],
        'classifier__max_depth': [None, 10, 20, 30]
    },
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.5, 0.7, 0.8, 0.90, 1],
        'classifier': [GaussianNB()]
    },
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.5, 0.7, 0.8, 0.90, 1],
        'classifier': [SVC(probability=True)],
        'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'classifier__C': [0.1, 1, 10, 100]
    },
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.5, 0.7, 0.8, 0.90, 1],
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [200, 300],
        'classifier__max_features': ['auto', 'sqrt', 'log2']
    },
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.5, 0.7, 0.8, 0.90, 1],
        'classifier': [LogisticRegression(max_iter=500)],
        'classifier__C': [0.01, 0.1, 1, 10, 100]
    }
]

# Definir Leave-One-Group-Out cross-validation
logo = LeaveOneGroupOut()

# Lista para almacenar los pipelines ajustados
pipelines = []

# Iterar sobre cada gesto único
for gesture in gestures:

    df_gesture = df[df['GestureLabel'] == gesture]
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = split_df_gestures(df_gesture, 'CorrectLabel')

    print(f'\n------------ CLASIFICACION GESTO {gesture} -----------\n')

    # Crear el pipeline con técnicas de balanceo, reducción de dimensión y modelo de clasificación
    pipeline = Pipeline([('balance_data',  SMOTETomek(smote=SMOTE(k_neighbors=2))),
                        ('reduce_dim', PCA()),
                        ('classifier', KNeighborsClassifier(n_neighbors=2))])

    # RandomizedSearchCVpara encontrar los mejores hiperparámetros
    random = RandomizedSearchCV(pipeline, param_grid, cv=logo, n_jobs=-1)
    random.fit(X_train, y_train, groups=X_train['SubjectID'])
    # Mejor pipeline encontrado
    best_pipeline = random.best_estimator_
    pipelines.append(best_pipeline)

    # Evaluar el modelo en el conjunto de entrenamiento y prueba
    print('Training set score: ' + str(best_pipeline.score(X_train, y_train)))
    print('Test set score: ' + str(best_pipeline.score(X_test, y_test)))
    print(best_pipeline)

    # Calcular la curva ROC
    y_prob = best_pipeline.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_prob, pos_label = 2) 
    roc_auc = auc(fpr, tpr)
    
    # Graficar la curva ROC
    plt.figure()  
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()